[link](https://adventofcode.com/2023/day/5)

In [1]:
YEAR = 2023 
DAY = 5

from aocd import get_data
import sys
sys.path.append("..")
from config import load_config
load_config()
data = get_data(year=YEAR, day=DAY)

# part 1

## problem statement

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

In [2]:
samp = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2
52 50 48
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```

## try 1 (fail)

In [3]:
dat = samp

In [4]:
dat.splitlines()[0].split(':')[1].strip()

'79 14 55 13'

In [5]:
numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
numbers

[79, 14, 55, 13]

In [6]:
def int_list(line: str):
    return [int(n) for n in line.strip().split(' ')]

In [7]:
line = '798 14 55 13'
int_list(line)

[798, 14, 55, 13]

In [8]:
xs = samp.splitlines()[1:]
xs

['',
 'seed-to-soil map:',
 '50 98 2',
 '52 50 48',
 '',
 'soil-to-fertilizer map:',
 '0 15 37',
 '37 52 2',
 '39 0 15',
 '',
 'fertilizer-to-water map:',
 '49 53 8',
 '0 11 42',
 '42 0 7',
 '57 7 4',
 '',
 'water-to-light map:',
 '88 18 7',
 '18 25 70',
 '',
 'light-to-temperature map:',
 '45 77 23',
 '81 45 19',
 '68 64 13',
 '',
 'temperature-to-humidity map:',
 '0 69 1',
 '1 0 69',
 '',
 'humidity-to-location map:',
 '60 56 37',
 '56 93 4']

In [9]:
all_ranges_str = []
i = 0 
while i < len(xs):
    if (len(xs[i]) == 0) or not xs[i][0].isdigit():
        print(f'{i}: not a map')
        i += 1
        continue    
    ranges_str = []
    while i < len(xs) and len(xs[i]) > 0 and xs[i][0].isdigit():
        print(f'{i}: map !')
        ranges_str.append(int_list(xs[i]))
        i += 1 
    print(f'ranges_str: {ranges_str}')
    all_ranges_str.append(ranges_str)

0: not a map
1: not a map
2: map !
3: map !
ranges_str: [[50, 98, 2], [52, 50, 48]]
4: not a map
5: not a map
6: map !
7: map !
8: map !
ranges_str: [[0, 15, 37], [37, 52, 2], [39, 0, 15]]
9: not a map
10: not a map
11: map !
12: map !
13: map !
14: map !
ranges_str: [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]]
15: not a map
16: not a map
17: map !
18: map !
ranges_str: [[88, 18, 7], [18, 25, 70]]
19: not a map
20: not a map
21: map !
22: map !
23: map !
ranges_str: [[45, 77, 23], [81, 45, 19], [68, 64, 13]]
24: not a map
25: not a map
26: map !
27: map !
ranges_str: [[0, 69, 1], [1, 0, 69]]
28: not a map
29: not a map
30: map !
31: map !
ranges_str: [[60, 56, 37], [56, 93, 4]]


In [10]:
def extract_ranges(xs: list[str]) -> list[tuple[int, int, int]]:
    all_ranges_str = []
    i = 0 
    while i < len(xs):
        if (len(xs[i]) == 0) or not xs[i][0].isdigit():
            i += 1
            continue    
        ranges_str = []
        while i < len(xs) and len(xs[i]) > 0 and xs[i][0].isdigit():
            ranges_str.append(int_list(xs[i]))
            i += 1 
        all_ranges_str.append(ranges_str)
    return all_ranges_str

In [11]:
xs = samp.splitlines()[1:]
extract_ranges(xs)

[[[50, 98, 2], [52, 50, 48]],
 [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 [[88, 18, 7], [18, 25, 70]],
 [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 [[0, 69, 1], [1, 0, 69]],
 [[60, 56, 37], [56, 93, 4]]]

In [12]:
def build_lookup_from_range(range_int: tuple[int, int, int]) -> dict[int, int]:
    ...

In [13]:
range_int = [88, 18, 7]
dst_start, src_start, le = range_int
lookup = {s:d for (s,d) in zip(range(src_start, src_start + le), range(dst_start, dst_start + le))}
lookup

{18: 88, 19: 89, 20: 90, 21: 91, 22: 92, 23: 93, 24: 94}

In [14]:
def build_lookup_from_range(range_int: tuple[int, int, int]) -> dict[int, int]:
    dst_start, src_start, le = range_int
    return {s:d for (s,d) in zip(range(src_start, src_start + le), range(dst_start, dst_start + le))}

In [15]:
range_int = [88, 18, 7]
build_lookup_from_range(range_int)

{18: 88, 19: 89, 20: 90, 21: 91, 22: 92, 23: 93, 24: 94}

In [16]:
def build_lookup_from_ranges(ranges_int: list[tuple[int, int, int]]) -> dict[int, int]:
    ...

In [17]:
ranges_int = [[50, 98, 2], [52, 50, 48]]
lookup = {}
for range_int in ranges_int:
    print(range_int)
    lookup.update(build_lookup_from_range(range_int))
lookup = {k: lookup[k] for k in sorted(lookup.keys())}
lookup

[50, 98, 2]
[52, 50, 48]


{50: 52,
 51: 53,
 52: 54,
 53: 55,
 54: 56,
 55: 57,
 56: 58,
 57: 59,
 58: 60,
 59: 61,
 60: 62,
 61: 63,
 62: 64,
 63: 65,
 64: 66,
 65: 67,
 66: 68,
 67: 69,
 68: 70,
 69: 71,
 70: 72,
 71: 73,
 72: 74,
 73: 75,
 74: 76,
 75: 77,
 76: 78,
 77: 79,
 78: 80,
 79: 81,
 80: 82,
 81: 83,
 82: 84,
 83: 85,
 84: 86,
 85: 87,
 86: 88,
 87: 89,
 88: 90,
 89: 91,
 90: 92,
 91: 93,
 92: 94,
 93: 95,
 94: 96,
 95: 97,
 96: 98,
 97: 99,
 98: 50,
 99: 51}

In [18]:
def build_lookup_from_ranges(ranges_int: list[tuple[int, int, int]], sort_keys: bool = True) -> dict[int, int]:
    ranges_int = [[50, 98, 2], [52, 50, 48]]
    lookup = {}
    for range_int in ranges_int:
        lookup.update(build_lookup_from_range(range_int))
    if sort_keys:
        lookup = {k: lookup[k] for k in sorted(lookup.keys())}
    return lookup

In [19]:
ranges_int = [[50, 98, 2], [52, 50, 48]]
build_lookup_from_ranges(ranges_int)

{50: 52,
 51: 53,
 52: 54,
 53: 55,
 54: 56,
 55: 57,
 56: 58,
 57: 59,
 58: 60,
 59: 61,
 60: 62,
 61: 63,
 62: 64,
 63: 65,
 64: 66,
 65: 67,
 66: 68,
 67: 69,
 68: 70,
 69: 71,
 70: 72,
 71: 73,
 72: 74,
 73: 75,
 74: 76,
 75: 77,
 76: 78,
 77: 79,
 78: 80,
 79: 81,
 80: 82,
 81: 83,
 82: 84,
 83: 85,
 84: 86,
 85: 87,
 86: 88,
 87: 89,
 88: 90,
 89: 91,
 90: 92,
 91: 93,
 92: 94,
 93: 95,
 94: 96,
 95: 97,
 96: 98,
 97: 99,
 98: 50,
 99: 51}

In [20]:
numbers = [int(n) for n in samp.splitlines()[0].split(':')[1].strip().split(' ')]
numbers

[79, 14, 55, 13]

In [21]:
numbers = [lookup.get(n, n) for n in numbers]
numbers

[81, 14, 57, 13]

In [22]:
dat = samp

numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
xs = dat.splitlines()[1:]
all_ranges_int = extract_ranges(xs)
all_ranges_int

[[[50, 98, 2], [52, 50, 48]],
 [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 [[88, 18, 7], [18, 25, 70]],
 [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 [[0, 69, 1], [1, 0, 69]],
 [[60, 56, 37], [56, 93, 4]]]

In [23]:
for ranges_int in all_ranges_int:
    lookup = build_lookup_from_ranges(ranges_int)
    numbers = [lookup.get(n, n) for n in numbers]
print(numbers)

[93, 14, 69, 13]


In [24]:
print(min(numbers))
assert min(numbers) == 13

13


In [25]:
def sol_2023_5_1(dat: str) -> int:
    numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
    xs = dat.splitlines()[1:]
    all_ranges_int = extract_ranges(xs)
    for ranges_int in all_ranges_int:
        lookup = build_lookup_from_ranges(ranges_int)
        numbers = [lookup.get(n, n) for n in numbers]
    return min(numbers)

In [26]:
sol_2023_5_1(samp)

13

In [27]:
#sol_2023_5_1(data)

This takes forever to compute --> fail.

It is inefficient to create a dict for lookups. We need to find another way

## try 2

We don't need to build a dictionary. For each number, we can directly compute the mapping given the range information

In [28]:
# what we keep
dat = samp 

numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]

xs = dat.splitlines()[1:]

def int_list(line: str):
    return [int(n) for n in line.strip().split(' ')]

def extract_ranges(xs: list[str]) -> list[tuple[int, int, int]]:
    all_ranges_str = []
    i = 0 
    while i < len(xs):
        if (len(xs[i]) == 0) or not xs[i][0].isdigit():
            i += 1
            continue    
        ranges_str = []
        while i < len(xs) and len(xs[i]) > 0 and xs[i][0].isdigit():
            ranges_str.append(int_list(xs[i]))
            i += 1 
        all_ranges_str.append(ranges_str)
    return all_ranges_str

In [29]:
nb = 99
ranges_int = [[50, 98, 2], [52, 50, 48]]

In [30]:
nb2 = None
for range_int in ranges_int:
    dst, src, l = range_int 
    if nb >= src and nb < src + l:
        nb2 = dst + (nb - src)
if nb2 is None:
    nb2 = nb
nb2

51

In [31]:
def lookup_nb(nb: int, ranges_int: list[tuple[int, int, int]]) -> int:
    ...

In [32]:
def lookup_nb(nb: int, ranges_int: list[tuple[int, int, int]]) -> int:
    nb2 = None
    for range_int in ranges_int:
        dst, src, l = range_int 
        if nb >= src and nb < src + l:
            nb2 = dst + (nb - src)
    if nb2 is None:
        nb2 = nb
    return nb2

In [33]:
nb = 77
ranges_int = [[50, 98, 2], [52, 50, 48]]
lookup_nb(nb, ranges_int)

79

In [34]:
dat = samp 

numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
xs = dat.splitlines()[1:]
all_ranges_int = extract_ranges(xs)

In [35]:
for ranges_int in all_ranges_int:
    numbers = [lookup_nb(n, ranges_int) for n in numbers]
print(numbers)
print(min(numbers))

[82, 43, 86, 35]
35


In [36]:
def sol_2023_5_1(dat: str) -> int:
    numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
    xs = dat.splitlines()[1:]
    all_ranges_int = extract_ranges(xs)
    for ranges_int in all_ranges_int:
        numbers = [lookup_nb(n, ranges_int) for n in numbers]
    return min(numbers)

In [37]:
sol_2023_5_1(samp)

35

In [38]:
sol_2023_5_1(data)

57075758

# Part 2 

## Problem statement

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

## Process

### try 1 (fail)

In [39]:
# what we keep
dat = samp 

numbers = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]

xs = dat.splitlines()[1:]

def int_list(line: str):
    return [int(n) for n in line.strip().split(' ')]

def extract_ranges(xs: list[str]) -> list[tuple[int, int, int]]:
    all_ranges_str = []
    i = 0 
    while i < len(xs):
        if (len(xs[i]) == 0) or not xs[i][0].isdigit():
            i += 1
            continue    
        ranges_str = []
        while i < len(xs) and len(xs[i]) > 0 and xs[i][0].isdigit():
            ranges_str.append(int_list(xs[i]))
            i += 1 
        all_ranges_str.append(ranges_str)
    return all_ranges_str

def lookup_nb(nb: int, ranges_int: list[tuple[int, int, int]]) -> int:
    nb2 = None
    for range_int in ranges_int:
        dst, src, l = range_int 
        if nb >= src and nb < src + l:
            nb2 = dst + (nb - src)
    if nb2 is None:
        nb2 = nb
    return nb2

In [40]:
#assert answer == 46

In [41]:
def get_init_numbers(numbers_range: list[int]) -> list[int]:
    ...

In [42]:
numbers_range = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
numbers_range

[79, 14, 55, 13]

In [43]:
numbers_range = [79, 14, 55, 13, 1, 2]
nb_ranges = int(len(numbers_range) / 2)
nb_ranges

3

In [44]:
all_numbers = []
for i in range(nb_ranges):
    all_numbers += list(range(numbers_range[2*i], numbers_range[2*i] + numbers_range[2*i+1]))
all_numbers

[79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 1,
 2]

In [45]:
def get_init_numbers(numbers_range: list[int]) -> list[int]:
    nb_ranges = int(len(numbers_range) / 2)
    all_numbers = []
    for i in range(nb_ranges):
        all_numbers += list(range(numbers_range[2*i], numbers_range[2*i] + numbers_range[2*i+1]))
    return all_numbers 

In [46]:
numbers_range = [79, 14, 55, 13, 1, 2]
get_init_numbers(numbers_range)

[79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 1,
 2]

In [47]:
def sol_2023_5_2(dat: str) -> int:
    numbers_range = [int(n) for n in dat.splitlines()[0].split(':')[1].strip().split(' ')]
    numbers = get_init_numbers(numbers_range)
    print(numbers)
    return 
    xs = dat.splitlines()[1:]
    all_ranges_int = extract_ranges(xs)
    for ranges_int in all_ranges_int:
        numbers = [lookup_nb(n, ranges_int) for n in numbers]
    return min(numbers)

In [48]:
sol_2023_5_2(samp)

[79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]


In [49]:
#sol_2023_5_2(data)

Takes forever to compute: need to find a more efficient way

### try 2

To make it more efficient, I need to take advantage of the fact that most numbers to lookup are **adjacent**

Other approach: seeing it as **translations of integer intervals** performed in sequence

Example: 

In [50]:
seeds = '79 14 55 13'
seed_intervals = [[79, 79+13], [55, 55 + 12]]

In [51]:
seed_intervals

[[79, 92], [55, 67]]

In [52]:
# first lookup
"50 98 2"
[98, 98+1] # translated by (-98+50)

[98, 99]

intersection of [79, 92] and [98, 98+1] is null so it remains the same interval

itv1 
itv2

I want to divide itv1 using the intersection between itv1 and itv2


```
1
|-----------|
         
2
   |-----|

   =

|-||-----|--| 
```


if: 
- itv1 = [a1, b1]
- itv2 = [a2, b2]
- itv1 INTERSECT itv2 = [i1, i2]

The interval is divided as :

[a1, i1-1], [i1, i2], [i2+1, b1]

let's build a function that gives the intersection between 2 intervals:

In [53]:
from typing import Optional
Itv = tuple[int, int]
def intersect(itv1: Itv, itv2: Itv) -> Optional[Itv]:
    ...

In [54]:
def intersect(itv1: Itv, itv2: Itv) -> Optional[Itv]:
    a1, b1 = itv1 
    a2, b2 = itv2 
    if b1 < a2 or b2 < a1:
        return None 
    i1 = max(a1, a2)
    i2 = min(b1, b2)
    return i1, i2

In [55]:
itv1 = [5,10]
itv2 = [8,15]
intersect(itv1, itv2)

(8, 10)

In [56]:
## toy example for each cases
# case 1
itv1 = [5,10]
itv2 = [8,15]
out = [[5,7],[8,10]]

# case 2
itv1 = [5,10]
itv2 = [2,9]
out = [[5,9],[10]]

# case 3
itv1 = [5,10]
itv2 = [4,20]
out = [[5,10]]

# case 4
itv1 = [5,10]
itv2 = [6,8]
out = [[5], [6,8], [9,10]]


In [57]:
def cut_interval(itv1: tuple[int, int], itv2: tuple[int, int]) -> list[tuple[int, int]]:
    ...

In [58]:
def cut_interval(itv1: tuple[int, int], itsect: Optional[tuple[int, int]]) -> list[tuple[int, int]]:

    a1, b1 = itv1 

    if itsect is None :
        return [itv1]
    i1, i2 = itsect

    intervals = []
    
    if i1 >= (a1 + 1):
        intervals.append((a1, i1-1))
    
    intervals.append(itsect)

    if (i2 + 1) <= b1:
        intervals.append((i2+1, b1))
    
    return intervals


In [59]:
itv1 = [5,10]
itv2 = [8,15]
out = [[5,7],[8,10]]
cut_interval(itv1, itv2)

[(5, 7), [8, 15]]

In [60]:
# case 2
itv1 = [5,10]
itv2 = [2,9]
cut_interval(itv1, itv2)

[[2, 9], (10, 10)]

In [61]:
# case 3
itv1 = [5,10]
itv2 = [4,20]
cut_interval(itv1, itv2)

[[4, 20]]

In [62]:
# case 4
itv1 = [5,10]
itv2 = [6,8]
cut_interval(itv1, itv2)

[(5, 5), [6, 8], (9, 10)]

In [63]:
def translate(itv: Itv, bias: int):
    return [itv[0] + bias, itv[1] + bias]

In [64]:
itv = [4,8]
translate(itv, -3)

[1, 5]

transform a range mapping into an interval + bias:

In [65]:
def convert_mapping(map: tuple[int, int, int]) -> tuple[Itv, int]:
    ...

In [66]:
map = "50 98 2"
int_list(map)

[50, 98, 2]

In [67]:
itv = [int_list(map)[1], int_list(map)[1] + int_list(map)[2] -1] 
itv

[98, 99]

In [68]:
bias = -int_list(map)[1] + int_list(map)[0]
bias

-48

In [69]:
def convert_mapping(map: tuple[int, int, int]) -> tuple[Itv, int]:
    itv = [map[1], map[1] + map[2]-1] 
    bias = -map[1] + map[0]
    return (itv, bias)

In [70]:
map = [50, 98, 2]
map_fmt = convert_mapping(map)
map_fmt

([98, 99], -48)

In [71]:
itv1 = [79, 79 + 13]
itv2 = map_fmt[0]
bias = map_fmt[1]
itsect = intersect(itv1, itv2)

print(itv1)
print(itv2)
print(itsect)
print(cut_interval(itv1, itsect))
print(bias)

[79, 92]
[98, 99]
None
[[79, 92]]
-48


In [72]:
itsect = intersect(itv1, itv2)
cuts = cut_interval(itv1, itsect)
print(itsect)
print(cuts)

None
[[79, 92]]


In [73]:
cuts = [(a+bias,b+bias) if (a,b) == itsect else (a,b) for (a,b) in cuts]
print(cuts)

[(79, 92)]


In [74]:
def transform_itv(itv1: Itv, itv2: Itv, bias: int) -> tuple[list[Itv], list[Itv]]:
    itsect = intersect(itv1, itv2)
    cuts = cut_interval(itv1, itsect)
    transformed = [(a+bias,b+bias) for (a,b) in cuts if (a,b) == itsect]
    same = [(a,b) for (a,b) in cuts if (a,b) != itsect]
    
    return (transformed, same)

In [75]:
itv1 = [79, 79 + 13]
itv2 = map_fmt[0]
bias = map_fmt[1]
transform_itv(itv1, itv2, bias)

([], [(79, 92)])

In [76]:
init_str = "seeds: 79 14 55 13"
init_str2 = init_str.split(':')[1].strip()
print(init_str2)

79 14 55 13


In [77]:
init_int = int_list(init_str2)
init_int

[79, 14, 55, 13]

In [78]:
def init_intervals(str_init: str) -> list[Itv]:
    ...

In [79]:
nb_init_itv = int(len(init_int) / 2)
[[init_int[2*i], init_int[2*i] + init_int[2*i+1]-1] for i in range(nb_init_itv)]

[[79, 92], [55, 67]]

In [80]:
def init_intervals(init_str: str) -> list[Itv]:
    init_str2 = init_str.split(':')[1].strip()
    init_int = int_list(init_str2)
    nb_init_itv = int(len(init_int) / 2)
    return [[init_int[2*i], init_int[2*i] + init_int[2*i+1]-1] for i in range(nb_init_itv)]

In [81]:
init_str = "seeds: 79 14 55 13"
init_intervals(init_str)

[[79, 92], [55, 67]]

In [82]:
dat = samp
print(dat)

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4



In [83]:
init_str = dat.splitlines()[0]
xs = dat.splitlines()[1:]

intervals = init_intervals(init_str)
print(intervals)

[[79, 92], [55, 67]]


In [84]:
mappings = extract_ranges(xs)

In [85]:
converted_mappings = [[convert_mapping(map) for map in maps] for maps in mappings]
converted_mappings

[[([98, 99], -48), ([50, 97], 2)],
 [([15, 51], -15), ([52, 53], -15), ([0, 14], 39)],
 [([53, 60], -4), ([11, 52], -11), ([0, 6], 42), ([7, 10], 50)],
 [([18, 24], 70), ([25, 94], -7)],
 [([77, 99], -32), ([45, 63], 36), ([64, 76], 4)],
 [([69, 69], -69), ([0, 68], 1)],
 [([56, 92], 4), ([93, 96], -37)]]

In [86]:
intervals

[[79, 92], [55, 67]]

In [87]:
itv1 = intervals[1]
itv1

[55, 67]

In [88]:
itv2, bias = converted_mappings[0][1]
print(itv2)
print(bias)

[50, 97]
2


In [89]:
transform_itv(itv1, itv2, bias)

([(57, 69)], [])

In [90]:
dat = samp
init_str = dat.splitlines()[0]
xs = dat.splitlines()[1:]

intervals = init_intervals(init_str)
intervals

[[79, 92], [55, 67]]

In [91]:
mappings = extract_ranges(xs)
converted_mappings = [[convert_mapping(map) for map in maps] for maps in mappings]
for maps in converted_mappings:
    print(maps)

[([98, 99], -48), ([50, 97], 2)]
[([15, 51], -15), ([52, 53], -15), ([0, 14], 39)]
[([53, 60], -4), ([11, 52], -11), ([0, 6], 42), ([7, 10], 50)]
[([18, 24], 70), ([25, 94], -7)]
[([77, 99], -32), ([45, 63], 36), ([64, 76], 4)]
[([69, 69], -69), ([0, 68], 1)]
[([56, 92], 4), ([93, 96], -37)]


In [92]:
itv = intervals[0]
print(itv)
maps = converted_mappings[0]
print(maps)

[79, 92]
[([98, 99], -48), ([50, 97], 2)]


In [93]:
m = []
same = [itv]
for map in maps:
    itv2, bias = map
    [transform_itv(s, itv2, bias) for s in same]

In [94]:
# intervals = init_intervals(init_str)
# for itv in intervals:
#     itv_out = []

#     for i,maps in enumerate(converted_mappings):
#         for map in maps:
#             itv2, bias = map
#             transformed, same = transform_itv(itv, itv2, bias)
#         mm = [map for map in maps if intersect(itv, map[0]) is not None]
#         assert len(mm) <= 1, i
#         if len(mm) == 0:
#             itv_out.append(itv)
#         else:
#             itv2, bias = mm[0]
#             itv_out.append(transform_itv(itv, itv2, bias))
    
# print(itv_out)


In [95]:
converted_mappings[4]

[([77, 99], -32), ([45, 63], 36), ([64, 76], 4)]

## new try

I assume that at each step, mappings don't intersect (otherwise the result would change depending on the order in which mappings are applied)

If so, for each input interval I can iterate through all mappings and return only the TRANSFORMED PART (corresponding to the intersection)

At the end, I also need to add the parts of the interval that WERE NOT AFFECTED, by "removing" each intersection

In [96]:
## functions to keep 

def int_list(line: str):
    return [int(n) for n in line.strip().split(' ')]

def init_intervals(dat: str) -> list[Itv]:
    init_str = dat.splitlines()[0]
    init_str2 = init_str.split(':')[1].strip()
    init_int = int_list(init_str2)
    nb_init_itv = int(len(init_int) / 2)
    return [[init_int[2*i], init_int[2*i] + init_int[2*i+1]-1] for i in range(nb_init_itv)]


def convert_mapping(map: tuple[int, int, int]) -> tuple[Itv, int]:
    itv = [map[1], map[1] + map[2]-1] 
    bias = -map[1] + map[0]
    return (itv, bias)


def extract_ranges(dat: str) -> list[tuple[int, int, int]]:
    xs = dat.splitlines()[1:]
    all_ranges = []
    i = 0 
    while i < len(xs):
        if (len(xs[i]) == 0) or not xs[i][0].isdigit():
            i += 1
            continue    
        ranges_raw = []
        while i < len(xs) and len(xs[i]) > 0 and xs[i][0].isdigit():
            ranges_raw.append(int_list(xs[i]))
            i += 1 
        ranges_cvt = [convert_mapping(r) for r in ranges_raw]
        all_ranges.append(ranges_cvt)
    return all_ranges


def intersect(itv1: Itv, itv2: Itv) -> Optional[Itv]:
    a1, b1 = itv1 
    a2, b2 = itv2 
    if b1 < a2 or b2 < a1:
        return None 
    i1 = max(a1, a2)
    i2 = min(b1, b2)
    return i1, i2

In [97]:
dat = samp
print(dat)

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4



In [98]:
intervals = init_intervals(dat)
intervals

[[79, 92], [55, 67]]

In [ ]:
mappings = extract_ranges(dat)
mappings

[[([98, 99], -48), ([50, 97], 2)],
 [([15, 51], -15), ([52, 53], -15), ([0, 14], 39)],
 [([53, 60], -4), ([11, 52], -11), ([0, 6], 42), ([7, 10], 50)],
 [([18, 24], 70), ([25, 94], -7)],
 [([77, 99], -32), ([45, 63], 36), ([64, 76], 4)],
 [([69, 69], -69), ([0, 68], 1)],
 [([56, 92], 4), ([93, 96], -37)]]

In [100]:
itv1 = intervals[0]
print(itv1)
map = mappings[0][1]
itv2 = map[0]
bias = map[1]
print(map)

[79, 92]
([50, 97], 2)


In [101]:
a, b = intersect(itv1, itv2)
(a+bias, b+bias)

(81, 94)

In [102]:
def transform_intersection_with_map(itv: Itv, map: tuple[Itv, int]) -> list[Itv]:
    itv_map = map[0]
    bias = map[1]
    its = intersect(itv, itv_map)
    if its is None:
        return []
    a,b = its
    return [(a+bias, b+bias)]

In [103]:
itv1 = (10, 20)
map = [(3, 13), 2]
transform_intersection_with_map(itv1, map)

[(12, 15)]

Now we need a function to remove from itv1 its intersection with itv2 

In [104]:
def remove_intersection_with_map(itv: Itv, map: tuple[Itv, int]) -> list[Itv]:
    ...

In [105]:
itv = (10, 20)
map = [(11, 13), 2]
itv_map = map[0]
its = intersect(itv, itv_map)
its

(11, 13)

In [106]:
a, b = itv 
if its is None:
    res = [itv]
else:
    i1, i2 = its
    res = []
    if i1 >= (a+1):
        res.append((a, i1-1))
    if i2 <= (b-1):
        res.append((i2+1, b))
res

[(10, 10), (14, 20)]

In [160]:
def remove_intersection_with_map(itv: Itv, map: tuple[Itv, int]) -> list[Itv]:
    itv_map = map[0]
    its = intersect(itv, itv_map)
    a, b = itv
    if its is None:
        res = [itv]
    elif its == itv:
        res = []
    else:
        i1, i2 = its
        res = []
        if i1 >= (a+1):
            res.append((a, i1-1))
        if i2 <= (b-1):
            res.append((i2+1, b))
    return res

In [161]:
itv = (12, 13)
map = [(12, 13), 2]
remove_intersection_with_map(itv, map)

[]

In [162]:
itv = (2, 4)
map = [(12, 13), 2]
remove_intersection_with_map(itv, map)

[(2, 4)]

In [163]:
itv = (10, 14)
map = [(12, 13), 2]
remove_intersection_with_map(itv, map)

[(10, 11), (14, 14)]

let's see how we gonna loop now

In [164]:
itv = (45, 110)
print(itv)
mapping = mappings[0]
mapping

(45, 110)


[([98, 99], -48), ([50, 97], 2)]

In [165]:
res = []
for map in mapping:
    res += transform_intersection_with_map(itv, map)
res

[(50, 51), (52, 99)]

In [166]:
res2 = [itv]
for map in mapping:
    res2 = [itv1 for elm in res2 for itv1 in remove_intersection_with_map(elm, map)]
res2 

[(45, 49), (100, 110)]

In [167]:
Map = tuple[Itv, bias]
def remove_intersection_with_mappings(itv: Itv, mappings: list[Map]):
    ...

In [168]:
def remove_intersection_with_mapping(itv: Itv, mappings: list[Map]):
    res2 = [itv]
    for map in mappings:
        res2 = [itv1 for elm in res2 for itv1 in remove_intersection_with_map(elm, map)]
    return res2 

In [169]:
itv = (45, 110)
print(itv)
mapping = mappings[0]
print(mapping)
remove_intersection_with_mapping(itv, mapping)

(45, 110)
[([98, 99], -48), ([50, 97], 2)]


[(45, 49), (100, 110)]

In [170]:
def pass_through_mapping(itv: Itv, mapping: list[Map]) -> list[Itv]:
    res = []
    for map in mapping:
        res += transform_intersection_with_map(itv, map)
    res2 = remove_intersection_with_mapping(itv, mapping)
    out = sorted(res + res2, key = lambda x: x[0]) 
    return out

In [171]:
itv = (10, 70)
print(itv)
mapping = mappings[1]
print(mapping)
pass_through_mapping(itv, mapping)

(10, 70)
[([15, 51], -15), ([52, 53], -15), ([0, 14], 39)]


[(0, 36), (37, 38), (49, 53), (54, 70)]

In [172]:
def merge_intervals(intervals):
    if not intervals:
        return []
    if len(intervals) == 1:
        return intervals
    
    intervals.sort(key=lambda x: x[0])
    
    merged = []
    current_start, current_end = intervals[0]
    
    for start, end in intervals[1:]:
        if start <= current_end + 1:
            current_end = max(current_end, end)
        else:
            merged.append((current_start, current_end))
            current_start, current_end = start, end
    
    if not merged or merged[-1][1] < current_end:
        merged.append((current_start, current_end))
    
    return merged

In [173]:
out = [(0, 36), (37, 38), (49, 53), (54, 70)]
merge_intervals(out)

[(0, 38), (49, 70)]

In [174]:
def pass_through_mapping(itv: Itv, mapping: list[Map]) -> list[Itv]:
    res = []
    for map in mapping:
        res += transform_intersection_with_map(itv, map)
    res2 = remove_intersection_with_mapping(itv, mapping)
    out = sorted(res + res2, key = lambda x: x[0]) 
    out = merge_intervals(out)
    return out

In [175]:
itv = (0, 30)
print(itv)
mapping = mappings[2]
print(mapping)
pass_through_mapping(itv, mapping)

(0, 30)
[([53, 60], -4), ([11, 52], -11), ([0, 6], 42), ([7, 10], 50)]


[(0, 19), (42, 48), (57, 60)]

In [176]:
dat = samp
intervals = init_intervals(dat)
mappings = extract_ranges(dat)
intervals_out = []

for itv in intervals:
    itvs = [itv]
    for mapping in mappings:
        itvs = [itv2 for itv1 in itvs for itv2 in pass_through_mapping(itv1, mapping)]
    intervals_out += itvs
intervals_out = merge_intervals(intervals_out)
intervals_out

[(46, 60), (82, 84), (86, 89), (94, 98)]

In [180]:
print(intervals_out[0][0])

46


In [181]:
def sol_2023_5_2(dat: str) -> int:
    intervals = init_intervals(dat)
    mappings = extract_ranges(dat)
    intervals_out = []

    for itv in intervals:
        itvs = [itv]
        for mapping in mappings:
            itvs = [itv2 for itv1 in itvs for itv2 in pass_through_mapping(itv1, mapping)]
        intervals_out += itvs
    intervals_out = merge_intervals(intervals_out)
    return intervals_out[0][0]


In [182]:
dat = samp
sol_2023_5_2(dat)

46

In [183]:
dat = data
sol_2023_5_2(dat)

31161857